In [1]:
%pylab inline
import matplotlib.pyplot as plt
import gala
import gala.potential as gp
import gala.dynamics as gd

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
def hernquist_density(r, M, a):
    return M*a / (2*np.pi) / (r*(r+a)**3)

def flattened_hernquist_density(x, y, z, M, a, b, c, rs):
    s = np.sqrt((x/a)**2 + (y/b)**2 + (z/c)**2)
    return hernquist_density(s, M, rs)

In [4]:
M = 1.
rs = 1.
a = 1
b = 0.5
c = 0.5
coeff = gp.scf.compute_coeffs(flattened_hernquist_density, nmax=8, lmax=8,
                            M=M, r_s=a, args=(M,a, b, c, rs), skip_m=True)
(S,Serr),(T,Terr) = coeff

In [6]:
np.shape(S)

(9, 9, 9)